In [362]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance

In [363]:
train = pd.read_csv('dataset/development_dataset.csv')
test = pd.read_csv('dataset/leaderboard_dataset.csv')

In [364]:
train['VAR14'] = train['VAR14'].replace('.',1)
train['VAR21'] = train['VAR21'].replace({'Low','Medium','High'},(0,1,2))

In [365]:
train['VAR14']=train['VAR14'].astype(str).astype(int)
train['VAR21']=train['VAR21'].astype(str).astype(int)

In [366]:
train['VARS1'] =(train['VAR7'] - train['VAR16']).abs()
test['VARS1'] = (test['VAR7'] - test['VAR16']).abs()

In [367]:
columns = ['VAR15','VAR18','VAR7','VAR1']
train = train.drop(columns, axis=1)
test = test.drop(columns,axis=1)

In [368]:
# corr = train.corr()
# corr.style.background_gradient(cmap='coolwarm').set_precision(2)

In [369]:
X = train.drop(['VAR21'],axis=1)
Y = train['VAR21']

In [370]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=25)

In [371]:
D_train = xgb.DMatrix(X, label=Y)
D_test = xgb.DMatrix(X_test,label= Y_test)

In [372]:
param = {
    'eta': 0.1, 
    'max_depth': 9,  
    'objective': 'multi:softmax',
    'min child wt' :8,
    'gamma': 5,
    'num_class': 3} 

In [373]:
model = xgb.train(param,D_train, 70)

In [374]:
preds = model.predict(D_test)

In [375]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, preds, labels=[0,1,2])

array([[  98,  561,  125],
       [  11, 3140,  449],
       [  10, 1321, 1085]], dtype=int64)

In [376]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, preds)

0.6357352941176471

In [377]:
test['VAR14'] = test['VAR14'].replace('.',1)
test['VAR14']=test['VAR14'].astype(str).astype(int)

In [378]:
D_test1 = xgb.DMatrix(test)

In [379]:
prediction = model.predict(D_test1)

In [380]:
test = pd.read_csv('dataset/leaderboard_dataset.csv')

In [381]:
output = pd.DataFrame()
output['VAR1'] = test['VAR1']
output['VAR21']=prediction

In [382]:
output['VAR21'] = output['VAR21'].replace((0,1,2),{'Low','Medium','High'})

In [383]:
output.to_csv(r'Neil_hanna_IITKGP_1.csv', header = None, index=False)